In [1]:
import random
import pandas as pd
import time
from bs4 import BeautifulSoup
from selenium import webdriver

In [3]:
asins = pd.read_csv('ASIN.csv',header=None,index_col=False, usecols=[1])
asin = pd.Series(asins[1]).tolist()
x = random.choice(asin)
y = random.choice(asin)
test = asin[:10]
print('Length of ASIN list is: ' + str(len(asin)))
print('Test ASIN is: ' + x + ' ' + y)
print(test)

Length of ASIN list is: 2009
Test ASIN is: B08XQSSZHM B07G2ZF6GD
['B097QK6W46', 'B0911MSYGW', 'B0773ZY26F', 'B092XF3W9T', 'B078HMK7S3', 'B08BF4CZSV', 'B098YKNVWR', 'B072M34RQC', 'B0863881T1', 'B08LLD2QXJ']


In [4]:
%%time
list_of_dicts= []
for i in test:
    product_info = {}
    try:
        
        web = r"https://www.amazon.com/exec/obidos/ASIN/" + i
        path = "C:\\Users\\wiatr\\anaconda3\\chromedriver.exe"
        driver = webdriver.Chrome(path)
        driver.get(web)
        time.sleep(10)
    except:
        continue

    try:
        name = driver.find_element_by_xpath('//span[@id="productTitle"]')
        product_info['name'] = name.text.strip()
    except:
        product_info['name'] = 0
            
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        product_info['price'] = [elem.text for elem in soup.find_all('span', {'class': 'a-offscreen'}) if "$" in elem.text][0]
    except:
        product_info['price'] = 0
    
    try:
        table_names = ['productDetails_detailBullets_sections1','productDetails_detailBullets_sections2','productDetails_techSpec_section_1','productDetails_techSpec_section_2']
        for table in table_names:
            try:
                names = driver.find_elements_by_xpath('//table[@id="' + table + '"]/tbody/tr/th')
                values = driver.find_elements_by_xpath('//table[@id="' + table + '"]/tbody/tr/td')
            except:
                continue

            for x in range(len(names)):
                name = names[x].text.strip()
                value = values[x].text.strip()
                product_info[name] = value
                    
    except:
        pass
        
    list_of_dicts.append(product_info)  # Append scrape to dictionary
            
df = pd.DataFrame(list_of_dicts)
df.to_csv('output.csv')

driver.close()
driver.quit()

Wall time: 4min 48s
